# Reflection Pattern

The first pattern we are going to implement is the **reflection pattern**. 

This pattern allows the LLM to reflect and critique its outputs, following the next steps:

1. The LLM **generates** a candidate output. If you look at the diagram above, it happens inside the **"Generate"** box.
2. The LLM **reflects** on the previous output, suggesting modifications, deletions, improvements to the writing style, etc.
3. The LLM modifies the original output based on the reflections and another iteration begins ...

**Now, we are going to build, from scratch, each step, so that you can truly understand how this pattern works.**

## Generation Step

The first thing we need to consider is:

> What do we want to generate? A poem? An essay? Python code?

For this example, I've decided to test the Python coding skills of Llama3 70B (that's the LLM we are going to use for all the tutorials). In particular, we are going to ask our LLM to code a famous sorting algorithm: **Merge Sort**. 

---

### Groq Client and relevant imports

In [2]:
import os
from pprint import pprint
from groq import Groq
from dotenv import load_dotenv
from IPython.display import display_markdown

# Remember to load the environment variables. You should have the Groq API Key in there :)
load_dotenv()

client = Groq()

We will start the **"generation"** chat history with the system prompt, as we said before. In this case, let the LLM act like a Python 
programmer eager to receive feedback / critique by the user.

In [3]:
generation_chat_history = [
    {
        "role": "system",
        "content": "You are a Python programmer tasked with generating high quality Python code."
        "Your task is to Generate the best content possible for the user's request. If the user provides critique," 
        "respond with a revised version of your previous attempt."
    }
]

Now, as the user, we are going to ask the LLM to generate an implementation of the **Merge Sort** algorithm. Just add a new message with the **user** role to the chat history.

In [4]:
generation_chat_history.append(
    {
        "role": "user",
        "content": "Generate a Python implementation of the Merge Sort algorithm"
    }
)

Let's generate the first version of the essay.

In [5]:
mergesort_code = client.chat.completions.create(
    messages=generation_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

generation_chat_history.append(
    {
        "role": "assistant",
        "content": mergesort_code
    }
)

In [6]:
display_markdown(mergesort_code, raw=True)

Here is a Python implementation of the Merge Sort algorithm:
```
def merge_sort(arr):
    """
    Sorts an array of integers using the Merge Sort algorithm.

    Time complexity: O(n log n)
    Space complexity: O(n)

    :param arr: The input array to be sorted
    :return: The sorted array
    """
    if len(arr) <= 1:
        return arr

    mid = len(arr) // 2
    left = arr[:mid]
    right = arr[mid:]

    left = merge_sort(left)
    right = merge_sort(right)

    return merge(left, right)


def merge(left, right):
    """
    Merges two sorted arrays into a single sorted array.

    :param left: The first sorted array
    :param right: The second sorted array
    :return: The merged sorted array
    """
    result = []
    while len(left) > 0 and len(right) > 0:
        if left[0] <= right[0]:
            result.append(left.pop(0))
        else:
            result.append(right.pop(0))
    result.extend(left)
    result.extend(right)
    return result
```
Here's an explanation of how the code works:

1. The `merge_sort` function takes an array as input and recursively divides it into two halves until each half has only one element.
2. The `merge` function takes two sorted arrays as input and merges them into a single sorted array.
3. The `merge` function uses a temporary array `result` to store the merged elements.
4. The `merge` function compares elements from both arrays and adds the smaller element to the `result` array.
5. Once one of the input arrays is empty, the remaining elements from the other array are appended to the `result` array.
6. The `merge_sort` function returns the sorted array by recursively calling itself on each half of the input array and then merging the results.

You can test this implementation using a sample array, like this:
```
arr = [5, 2, 8, 3, 1, 6, 4]
arr = merge_sort(arr)
print(arr)  # [1, 2, 3, 4, 5, 6, 8]
```
Let me know if you have any questions or need further clarification!

## Reflection Step

Now, let's allow the LLM to reflect on its outputs by defining another system prompt. This system prompt will tell the LLM to act as Andrej Karpathy, computer scientist and Deep Learning wizard.

>To be honest, I don't think the fact of acting like Andrej Karpathy will influence the LLM outputs, but it was fun :)

In [7]:
reflection_chat_history = [
    {
    "role": "system",
    "content": "You are Andrej Karpathy, an experienced computer scientist. You are tasked with generating critique and recommendations for the user's code",
    }
]

The user message, in this case,  is the essay generated in the previous step. We simply add the `mergesort_code` to the `reflection_chat_history`.

In [8]:
reflection_chat_history.append(
    {
        "role": "user",
        "content": mergesort_code
    }
)

Now, let's generate a critique to the Python code.

In [9]:
critique = client.chat.completions.create(
    messages=reflection_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

In [10]:
display_markdown(critique, raw=True)

The classic Merge Sort algorithm!

Overall, your implementation looks clean and correct. Here are some minor critiques and suggestions for improvement:

1. **Docstring formatting**: In the docstrings, the parameter descriptions and return types are not formatted according to Python conventions. You can use the triple quotes `"""..."""` to format the docstrings correctly.

2. **Type hints**: It's a good practice to add type hints for function parameters and return types. This makes the code more readable and helps with static type checking.

3. **Variable naming**: The variable names `left` and `right` are concise, but `mid_left` and `mid_right` might be more descriptive for the divided halves.

4. **Redundant comments**: Some of the comments, such as `# The merge function takes two sorted arrays as input and merges them into a single sorted array.`, are redundant and can be removed, as the code itself is self-explanatory.

5. **Error handling**: The current implementation assumes that the input `arr` is a list of integers. You might want to add some error handling to ensure that the input is a list and contains only integers. This can be done using `isinstance()` and `all()` functions.

Here's the refactored code incorporating the above suggestions:
```python
def merge_sort(arr: list[int]) -> list[int]:
    """
    Sorts a list of integers using the Merge Sort algorithm.

    Time complexity: O(n log n)
    Space complexity: O(n)

    :param arr: The input list to be sorted
    :return: The sorted list
    """
    if len(arr) <= 1:
        return arr

    mid = len(arr) // 2
    mid_left = arr[:mid]
    mid_right = arr[mid:]

    mid_left = merge_sort(mid_left)
    mid_right = merge_sort(mid_right)

    return merge(mid_left, mid_right)


def merge(left: list[int], right: list[int]) -> list[int]:
    """
    Merges two sorted lists into a single sorted list.

    :param left: The first sorted list
    :param right: The second sorted list
    :return: The merged sorted list
    """
    result = []
    while left and right:
        if left[0] <= right[0]:
            result.append(left.pop(0))
        else:
            result.append(right.pop(0))
    result.extend(left or right)  # Use `or` to handle the case where one list is empty
    return result
```
With these minor changes, the code looks more polished and maintainable.

Finally, we just need to add this *critique* to the `generation_chat_history`, in this case, as the `user` role.

In [11]:
generation_chat_history.append(
    {
        "role": "user",
        "content": critique
    }
)

## Generation Step (II)

In [12]:
essay = client.chat.completions.create(
    messages=generation_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

In [13]:
display_markdown(essay, raw=True)

I appreciate your feedback and suggestions! You're right on point with each of your critiques, and I'm glad you provided a refactored version of the code that addresses these issues.

Here are some additional thoughts and responses to your suggestions:

1. **Docstring formatting**: Thank you for pointing out the docstring formatting issues. I'll make sure to follow the Python conventions for docstrings going forward.

2. **Type hints**: I completely agree about the importance of type hints. Adding type hints makes the code more readable and helps with static type checking, as you mentioned.

3. **Variable naming**: Your suggestion about using more descriptive variable names, such as `mid_left` and `mid_right`, is a good one. In general, it's a good idea to use descriptive variable names to make the code more readable and easier to understand.

4. **Redundant comments**: You're right that some comments can be redundant and can be removed if the code is self-explanatory. I'll be more mindful of this in the future and try to strike a balance between providing helpful comments and avoiding unnecessary ones.

5. **Error handling**: Your suggestion about adding error handling to ensure the input is a list and contains only integers is a good one. I'll make sure to add input validation and error handling to make the code more robust.

Here's the updated code with the suggested improvements:
```python
def merge_sort(arr: list[int]) -> list[int]:
    """
    Sorts a list of integers using the Merge Sort algorithm.

    Time complexity: O(n log n)
    Space complexity: O(n)

    :param arr: The input list to be sorted
    :return: The sorted list
    """
    if not isinstance(arr, list) or not all(isinstance(x, int) for x in arr):
        raise ValueError("Input must be a list of integers")

    if len(arr) <= 1:
        return arr

    mid = len(arr) // 2
    mid_left = arr[:mid]
    mid_right = arr[mid:]

    mid_left = merge_sort(mid_left)
    mid_right = merge_sort(mid_right)

    return merge(mid_left, mid_right)


def merge(left: list[int], right: list[int]) -> list[int]:
    """
    Merges two sorted lists into a single sorted list.

    :param left: The first sorted list
    :param right: The second sorted list
    :return: The merged sorted list
    """
    result = []
    while left and right:
        if left[0] <= right[0]:
            result.append(left.pop(0))
        else:
            result.append(right.pop(0))
    result.extend(left or right)  # Use `or` to handle the case where one list is empty
    return result
```
Thank you again for your feedback and suggestions!

## And the iteration starts again ...

After **Generation Step (II)** the corrected Python code will be received, once again, by Karpathy. Then, the LLM will reflect on the corrected output, suggesting further improvements and the loop will go, over and over for a number **n** of total iterations.

> There's another possibility. Suppose the Reflection step can't find any further improvement. In this case, we can tell the LLM to output some stop string, like "OK" or "Good" that means the process can be stopped. However, we are going to follow the first approach, that is, iterating for a fixed number of times.

# At
Implement a practical case where we can implement this concept

in the next class we can call 2 or 3 person to present it